In [9]:
# https://rosettacode.org/wiki/Sorting_algorithms/Insertion_sort#Python

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Insertionsort Algorithm

In [10]:
def insertion_sort(L): # x^2
    for i in range(1, len(L)):
        j = i-1 
        key = L[i]
        while j >= 0 and L[j] > key:
           L[j+1] = L[j]
           j -= 1
        L[j+1] = key

## Frequency Count Method

In [11]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(insertion_sort)

  # create a list with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [12]:
resultados_com_menor_loss = []
repeat = 5
registros = []
unary_operators_list = ["log", "square", "cube", "sqrt", "round", "exp", "abs"]

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=unary_operators_list
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=unary_operators_list
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=unary_operators_list
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-5, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           9.037e+05  1.594e+01  y = x₀                                        2           4.106e+04  3.091e+00  y = square(x₀)                                4           8.233e+03  8.035e-01  y = square(x₀ * 0.92744)                      6           8.004e+03  1.410e-02  y = square((x₀ * 0.92744) + -0.29884)         13          7.465e+03  9.961e-03  y = (square(cube((-1.1394 + -0.88217) * -0.67987) + (x₀ * 1.69...    

In [13]:
registros_ = registros
file_pickle = 'dados_v2.pck'

In [14]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [15]:
def salvar_dados(dados, key, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)
  else:
    dados_exist = {}
    
  valor_original = dados_exist.get(key)
  if valor_original == None:
    dados_exist.update({key: [dados]})
  else:
    if isinstance(valor_original, list):
      valor_original.append(dados)
    else:
      dados_exist.update({key: [dados]})

  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)

caminho_arquivo = file_pickle
novos_dados = lista_melhor_valor[0:3] + [lista_melhor_valor[4]]

salvar_dados(novos_dados, 'insertionsort_N^2', caminho_arquivo)

In [16]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = file_pickle
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items():
  print('\u25CF', k)
  for index, item in enumerate(v):
    if index == len(v)-1:
       print('└─', item)
    else:
      print('├─', item)
  print('==========================')

Conteúdo do arquivo pickle:
● binary_search_logx
├─ [6, 1.2914093, 1.37589542429269, log(x0**6) + 5.574131]
└─ [5, 1.5291592, 2.19479656667366, log(x0)**2 + 13.966276]
● boyer_moore_M_plus_N
├─ [9, 1729031400.0, 0.1972007458935316, 752018330.701125*(0.00603869219560025*x0 + 0.00603869219560025*x1 - 1)**4 + 601415.75]
└─ [5, 15754614000.0, 0.3055240605113849, 3621132.2 - 28185.828*x1]
● bubblesort_N^2
├─ [4, 5910.4966, 3.4645076652491285, 1.5253483*x0**2]
└─ [4, 8542.308, 3.303255717418494, 1.5552218*x0**2]
● fibonacci_interative_N
├─ [3, 0.20818244, 3.412549040667958, 2.058653*x0]
└─ [3, 0.20818244, 3.412549040667958, 2.058653*x0]
● fibonacci_recursive_2^N
├─ [15, 111113150.0, 0.37865996607381686, x0**4*(0.0022606994*x0**2 - 0.714783)]
└─ [15, 77371830.0, 0.38657145558895856, x0**2*(x0**2*(0.16452532*x0 - 3.8766947) + 297.9323)]
● heapsort_NlogN
├─ [9, 0.24989581, 1.1851428541960574, 4*x0 - 1.4897935]
└─ [9, 0.24989575, 1.185128414350167, 4*x0 - 1.4897884]
● insertionsort_N^2
├─ [4, 12